In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
success_df = pd.read_csv('charity_data.csv')
success_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Generate our categorical variable list
success_cat = success_df.dtypes[success_df.dtypes == "object"].index.tolist()


In [4]:
# Check the number of unique values in each column
success_df[success_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [5]:
# Drop name classification and application type columns since they are irrelevant to our analysis
success_df = success_df.drop(['CLASSIFICATION','APPLICATION_TYPE','NAME'], axis='columns')
success_df

,EIN,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,Independent,ProductDev,Association,1,0,N,5000,1
1,10531628,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,10553066,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,Independent,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,996009318,Independent,ProductDev,Association,1,0,N,5000,0
34295,996010315,CompanySponsored,ProductDev,Association,1,0,N,5000,0
34296,996012607,CompanySponsored,Preservation,Association,1,0,N,5000,0
34297,996015768,Independent,ProductDev,Association,1,0,N,5000,1


In [6]:
# Generate our categorical variable list
success_cat = success_df.dtypes[success_df.dtypes == "object"].index.tolist()

In [7]:
# Check the number of unique values in each column
success_df[success_cat].nunique()

AFFILIATION               6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(success_df[success_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(success_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
success_df = success_df.merge(encode_df,left_index=True, right_index=True)
success_df = success_df.drop(success_cat,1)
success_df.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = success_df["IS_SUCCESSFUL"].values
X = success_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
len(X_train[0])

29

### Deep Learning Model

In [94]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 8)                 240       
_________________________________________________________________
dense_26 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 6         
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________


In [95]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [96]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 5s 176us/sample - loss: 1793855.1141 - accuracy: 0.5052
Epoch 2/100
25724/25724 [==============================] - 3s 107us/sample - loss: 8279.0872 - accuracy: 0.5067
Epoch 3/100
25724/25724 [==============================] - 3s 112us/sample - loss: 12755.2037 - accuracy: 0.5038
Epoch 4/100
25724/25724 [==============================] - 3s 110us/sample - loss: 12672.4297 - accuracy: 0.5331
Epoch 5/100
25724/25724 [==============================] - 3s 106us/sample - loss: 12195.9627 - accuracy: 0.5332
Epoch 6/100
25724/25724 [==============================] - 3s 108us/sample - loss: 2250.3295 - accuracy: 0.5327
Epoch 7/100
25724/25724 [==============================] - 3s 111us/sample - loss: 12791.6798 - accuracy: 0.5324
Epoch 8/100
25724/25724 [==============================] - 3s 117us/sample - loss: 4147.4413 - accuracy: 0.5326s - loss
Epoch 9/100
25724/25724 [==============================] - 3s 126u

KeyboardInterrupt: 

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.6899 - accuracy: 0.5332
Loss: 0.6909440315916656, Accuracy: 0.5331778526306152


### Attempts to improve model:
#### 1) Increase number of hidden layers

In [32]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 8)                 448       
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 6         
Total params: 529
Trainable params: 529
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 4s 166us/sample - loss: 832571.5857 - accuracy: 0.5163
Epoch 2/100
25724/25724 [==============================] - 3s 111us/sample - loss: 35879.0141 - accuracy: 0.5326
Epoch 3/100
25724/25724 [==============================] - 3s 111us/sample - loss: 11631.6435 - accuracy: 0.5326
Epoch 4/100
25724/25724 [==============================] - 3s 112us/sample - loss: 2086.7635 - accuracy: 0.5322
Epoch 5/100
25724/25724 [==============================] - 3s 110us/sample - loss: 416.7635 - accuracy: 0.5323
Epoch 6/100
25724/25724 [==============================] - 3s 118us/sample - loss: 27.3453 - accuracy: 0.5323
Epoch 7/100
25724/25724 [==============================] - 3s 115us/sample - loss: 14.6903 - accuracy: 0.5325
Epoch 8/100
25724/25724 [==============================] - 4s 169us/sample - loss: 8.1432 - accuracy: 0.5325
Epoch 9/100
25724/25724 [==============================] - 3s 133us/sample - loss: 0.

25724/25724 [==============================] - 3s 122us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 75/100
25724/25724 [==============================] - 3s 113us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 76/100
25724/25724 [==============================] - 3s 113us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 77/100
25724/25724 [==============================] - 3s 125us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 78/100
25724/25724 [==============================] - 3s 132us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 79/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 80/100
25724/25724 [==============================] - 4s 140us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 81/100
25724/25724 [==============================] - 4s 140us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 82/100
25724/25724 [==============================] - 3s 126us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 83/100
25724/2572

In [97]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.6899 - accuracy: 0.5332
Loss: 0.6909442992252094, Accuracy: 0.5331778526306152


By increasing the number of hidden layers both the accuracy and the loss is exactly the same as our original model, with an accuracy of 53.3% and a loss of 0.691.

#### 2) Increasing number of neurons

In [13]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  len(X_train[0])*2
hidden_nodes_layer2 = 8
#hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 58)                1740      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 472       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 2,221
Trainable params: 2,221
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 4s 167us/sample - loss: 629342.5156 - accuracy: 0.5068
Epoch 2/100
25724/25724 [==============================] - 3s 111us/sample - loss: 456783.1514 - accuracy: 0.5037
Epoch 3/100
25724/25724 [==============================] - 3s 107us/sample - loss: 442614.2153 - accuracy: 0.5025
Epoch 4/100
25724/25724 [==============================] - 3s 107us/sample - loss: 385963.4700 - accuracy: 0.5037
Epoch 5/100
25724/25724 [==============================] - 3s 109us/sample - loss: 351615.0079 - accuracy: 0.5036
Epoch 6/100
25724/25724 [==============================] - 4s 138us/sample - loss: 356192.6856 - accuracy: 0.5038
Epoch 7/100
25724/25724 [==============================] - 4s 139us/sample - loss: 355989.1504 - accuracy: 0.5070
Epoch 8/100
25724/25724 [==============================] - 3s 112us/sample - loss: 334954.0480 - accuracy: 0.5037
Epoch 9/100
25724/25724 [==============================] - 3s 101

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#### 3) Change Input data

In [98]:
success_df.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Remove special considerations and status input columns

In [142]:
# Split our preprocessed data into our features and target arrays
y = success_df["IS_SUCCESSFUL"].values
X1 = success_df.drop(["IS_SUCCESSFUL",'SPECIAL_CONSIDERATIONS_N' ,'SPECIAL_CONSIDERATIONS_Y','STATUS'],1).values

# Split the preprocessed data into a training and testing dataset
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X1, y, random_state=78)

In [143]:
# Create a StandardScaler instance
scaler_r = StandardScaler()

# Fit the StandardScaler
X_scaler_r = scaler_r.fit(X_train_r)

# Scale the data
X_train_scaler_1 = X_scaler_r.transform(X_train_r)
X_test_scaled_r = X_scaler_r.transform(X_test_r)

In [144]:
len(X_train_r[0])

26

In [145]:
# Define the model - deep neural net
number_input_features = len(X_train_r[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 8)                 216       
_________________________________________________________________
dense_46 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 6         
Total params: 267
Trainable params: 267
Non-trainable params: 0
_________________________________________________________________


In [124]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [125]:
# Train the model
fit_model = nn.fit(X_train_r,y_train_r,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 6s 227us/sample - loss: 757570.2741 - accuracy: 0.5005
Epoch 2/100
25724/25724 [==============================] - 3s 104us/sample - loss: 74624.5520 - accuracy: 0.5018
Epoch 3/100
25724/25724 [==============================] - 3s 99us/sample - loss: 63604.7068 - accuracy: 0.5046
Epoch 4/100
25724/25724 [==============================] - 3s 105us/sample - loss: 44938.0895 - accuracy: 0.4965
Epoch 5/100
25724/25724 [==============================] - 3s 106us/sample - loss: 51588.3239 - accuracy: 0.4986
Epoch 6/100
25724/25724 [==============================] - 3s 113us/sample - loss: 47206.5203 - accuracy: 0.4982
Epoch 7/100
25724/25724 [==============================] - 3s 128us/sample - loss: 50607.2571 - accuracy: 0.5043
Epoch 8/100
25724/25724 [==============================] - 3s 126us/sample - loss: 52930.1717 - accuracy: 0.5065
Epoch 9/100
25724/25724 [==============================] - 4s 142us/sampl

25724/25724 [==============================] - 3s 107us/sample - loss: 397.8906 - accuracy: 0.5110
Epoch 73/100
25724/25724 [==============================] - 3s 107us/sample - loss: 15.4434 - accuracy: 0.5339
Epoch 74/100
25724/25724 [==============================] - 3s 110us/sample - loss: 3.5826 - accuracy: 0.5295
Epoch 75/100
25724/25724 [==============================] - 3s 115us/sample - loss: 0.6977 - accuracy: 0.5321
Epoch 76/100
25724/25724 [==============================] - 3s 110us/sample - loss: 0.6917 - accuracy: 0.5321
Epoch 77/100
25724/25724 [==============================] - 3s 105us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 78/100
25724/25724 [==============================] - 3s 110us/sample - loss: 0.6911 - accuracy: 0.5321 - los
Epoch 79/100
25724/25724 [==============================] - 3s 109us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 80/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 81/100
2

In [126]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_r,y_test_r,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.6900 - accuracy: 0.5332
Loss: 0.6909448554147437, Accuracy: 0.5331778526306152


By changing the input data, the accuracy didn't change significantly its almost the same with a 53.38% accuaracy and a 0.6907 loss, whereas the original mosel had an accuracy of 53.32% and loss of 0.691.

#### Random Forest

In [158]:
# import dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.664
